<a href="https://colab.research.google.com/github/sparkysparo/NLP-mood-metaphors/blob/main/Emotion_to_Metaphor_Mood_Uplifter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mood-to-Metaphor Uplifter App**

# **SECTION 1: SETTING UP ENVIROMENT**


In [2]:
# Install required packages (run once)
!pip install gradio pandas transformers torch matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import random
import re
import json
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Tuple, Optional, List

In [4]:
# Load the dataset
emotion_df = pd.read_csv("emotion_map_full.csv")
emotion_map = emotion_df.set_index("emotion").to_dict(orient="index")

# **SECTION 2: CORE FUNCTIONALITY**

In [5]:
def get_emotion_response(emotion: str) -> Optional[Dict[str, str]]:
    """Get full response for a recognized emotion"""
    emotion = emotion.lower().strip()
    return emotion_map.get(emotion)

def get_fallback_response() -> Tuple[str, str, str]:
    """Response for unrecognized emotions"""
    fallback_quotes = [
        "Emotions are the language of the soul.",
        "All feelings are valid - even the confusing ones.",
        "This too shall pass."
    ]
    fallback_suggestions = [
        "Try journaling about how you feel.",
        "Take three deep breaths and check in with your body.",
        "Consider talking to someone about what you're experiencing."
    ]
    return (
        "Your feeling is complex and unique.",
        random.choice(fallback_quotes),
        random.choice(fallback_suggestions)
    )

def mood_to_response(user_input: str) -> Tuple[str, str, str]:
    """Core response function"""
    response = get_emotion_response(user_input)
    return response["metaphor"], response["quote"], response["suggestion"] if response else get_fallback_response()

# **SECTION 3: EMOTION CLASSIFICATION**

In [6]:
from transformers import pipeline

class EmotionClassifier:
    def __init__(self):
        self.classifier = pipeline("zero-shot-classification",
                                 model="facebook/bart-large-mnli")
        self.emotion_labels = list(emotion_map.keys())

    def classify_emotion(self, text: str) -> str:
        """Classify free-form text into one of our known emotions"""
        result = self.classifier(
            text,
            candidate_labels=self.emotion_labels,
            multi_label=False
        )
        return result['labels'][0]  # return the top prediction

emotion_classifier = EmotionClassifier()

def intelligent_mood_to_response(user_input: str) -> Tuple[str, str, str]:
    """Handle free-form text input"""
    primary_emotion = emotion_classifier.classify_emotion(user_input)
    return mood_to_response(primary_emotion)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


# **SECTION 4: DYNAMIC CONTENT GENERATION**

In [7]:
from transformers import pipeline, set_seed

class MetaphorGenerator:
    def __init__(self):
        self.generator = pipeline("text-generation", model="gpt2")
        set_seed(42)  # For reproducibility
        self.safety_filter = [
            "violence", "harm", "hate", "suicide", "kill"
        ]

    def generate_metaphor(self, emotion: str) -> str:
        prompt = f"A creative metaphor for {emotion} is"
        try:
            result = self.generator(
                prompt,
                max_length=25,
                num_return_sequences=1,
                temperature=0.7
            )
            metaphor = result[0]['generated_text'].replace(prompt, "").strip()
            metaphor = re.sub(r'[^a-zA-Z ,.\'-]', '', metaphor)

            # Safety check
            if any(bad_word in metaphor.lower() for bad_word in self.safety_filter):
                return emotion_map.get(emotion, {}).get("metaphor", "A feeling that's meaningful")
            return metaphor
        except:
            return "A feeling that's hard to describe"

metaphor_generator = MetaphorGenerator()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


# **SECTION 5: EMOTION TRACKING**

In [8]:
class EmotionTracker:
    def __init__(self, storage_path: str = "emotion_history.json"):
        self.storage_path = Path(storage_path)
        self.history = []
        self.load_history()

    def load_history(self):
        if self.storage_path.exists():
            with open(self.storage_path, 'r') as f:
                self.history = json.load(f)

    def save_history(self):
        with open(self.storage_path, 'w') as f:
            json.dump(self.history, f, indent=2)

    def record_emotion(self, emotion: str):
        entry = {
            "emotion": emotion.lower(),
            "timestamp": datetime.now().isoformat()
        }
        self.history.append(entry)
        self.save_history()

    def get_stats(self, days: int = 7) -> Dict:
        """Get weekly emotion statistics"""
        cutoff = datetime.now() - timedelta(days=days)
        recent = [e for e in self.history
                 if datetime.fromisoformat(e["timestamp"]) >= cutoff]

        if not recent:
            return {}

        counts = {}
        for entry in recent:
            emotion = entry["emotion"]
            counts[emotion] = counts.get(emotion, 0) + 1

        return {
            "total": len(recent),
            "most_common": max(counts.items(), key=lambda x: x[1]),
            "distribution": counts
        }

emotion_tracker = EmotionTracker()

# **SECTION 6: GRADIO INTERFACE**

In [9]:
import gradio as gr

def visualize_emotion(emotion: str):
    """Simple visualization for recognized emotions"""
    if emotion not in emotion_map:
        return None

    color_map = {
        "sadness": "blue",
        "joy": "gold",
        "anger": "red",
        "anxiety": "purple",
        "fear": "darkred",
        "love": "pink",
        "hope": "lightblue",
        "peace": "green"
    }

    color = color_map.get(emotion, "gray")
    fig, ax = plt.subplots(figsize=(6, 1))
    gradient = np.linspace(0, 1, 256).reshape(1, -1)
    gradient = np.vstack((gradient, gradient))

    ax.imshow(gradient, aspect='auto', cmap=plt.cm.colors.ListedColormap([
        (to_rgba(color)[0], to_rgba(color)[1], to_rgba(color)[2], 0.3),
        (to_rgba(color)[0], to_rgba(color)[1], to_rgba(color)[2], 0.8)
    ]))
    ax.set_title(emotion.capitalize(), color='white', pad=10)
    ax.set_axis_off()
    return fig

def full_response(user_input: str):
    """Handle complete request flow"""
    # Classify and track
    emotion = emotion_classifier.classify_emotion(user_input)
    emotion_tracker.record_emotion(emotion)

    # Get response (30% chance of dynamic metaphor)
    if random.random() < 0.3:
        metaphor = metaphor_generator.generate_metaphor(emotion)
        quote = emotion_map.get(emotion, {}).get("quote", "This feeling matters.")
        suggestion = emotion_map.get(emotion, {}).get("suggestion", "Be kind to yourself.")
    else:
        metaphor, quote, suggestion = mood_to_response(emotion)

    # Visualization
    viz = visualize_emotion(emotion)

    return metaphor, quote, suggestion, viz

# Create interface
with gr.Blocks(title="Mood Companion", theme="soft") as app:
    gr.Markdown("""
    # Mood-to-Metaphor Companion
    *Transform your emotions into understanding*
    """)

    with gr.Row():
        with gr.Column():
            inp = gr.Textbox(label="How are you feeling?",
                            placeholder="e.g. 'joy', 'anxiety', or describe your mood...")
            btn = gr.Button("Get Insight", variant="primary")

        with gr.Column():
            metaphor = gr.Textbox(label="Metaphor")
            quote = gr.Textbox(label="Wisdom")
            suggestion = gr.Textbox(label="Suggestion")
            viz = gr.Plot(label="Emotion Color")

    examples = gr.Examples(
        examples=["I'm so excited!", "Feeling really down today", "Why am I so angry?"],
        inputs=inp
    )

    btn.click(
        fn=full_response,
        inputs=inp,
        outputs=[metaphor, quote, suggestion, viz]
    )


# **SECTION 7: RUNING THE APP LOCALLY**


In [10]:
if __name__ == "__main__":



    # To run locally
    app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2651158d980f14f6ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
